# Unit testing and cadCAD objects

On this notebook, I'll show some ideas and thoughts about unit testing and wrapping user functions / objects into cadCAD.

## Unit testing

### doctest

Maybe the most straightforward manner to include unit testing on cadCAD.

*Pros*
* Available on Python standard library
* Easy to learn and implement
* Straightforward and runs on jupyter notebooks
* Forces us to create docstrings

*Cons*
* Can generate clutter
* A lot of clutter if we want to do multiple tests
* Doesn't have many options for extending it

In [3]:
import doctest

def p_logistic_policy(params, substep, state_history, prev_state):
    """
    Yet another logistic map.
    
    >>> params = {'control_parameter': 3.7}
    >>> prev_state = {'population': 0.7}
    >>> x = p_logistic_policy(params, None, None, prev_state)
    >>> abs(x['replace_population'] - 0.777) < 0.001
    True
    """
    a = params['control_parameter']
    x = prev_state['population']
    new_pop = min(max(a * x * (1 - x), 0), 1)
    return {'replace_population': new_pop}


def s_population(params, substep, state_history, prev_state, policy_input):
    """
    Update population
    
    >>> policy_input = {'replace_population': 0.7}
    >>> s_population(None, None, None, None, policy_input)
    ('population', 0.7)
    """
    new_pop = policy_input.get('replace_population')
    return ('population', new_pop)

doctest.testmod()

TestResults(failed=0, attempted=6)

### Pytest

The most complete testing suite for Python. It can parse doctest strings as well as having separate and modular tests.

*Pros*
* Extensible and configurable
* Useful for complex applications
* Can reuse doctest strings

*Cons*
* Not so very straightforward
* Doesn't run on notebooks

### What I think

* $[doctest, pytest] \neq 0$
* `pytest(doctest(source))` runs okay, but `doctest(pytest(source))` gives error
* Doctest is quicker to prototype, but pytest is better for handling a lot of tests

## Object-oriented cadCAD objects

If we created objects for representing cadCAD data, logic and structure, we would have a lot of options for thinking how to plug simulations into user objects and classes. Also, the component code relation with the structure would be more visible.

In [1]:
# Wrapping everything
Configuration = lambda *args, **kwargs: None

# Structure
TimestepBlock = lambda *args, **kwargs: None
PartialStateUpdateBlock = lambda *args, **kwargs: None

# Logic
Policy = lambda *args, **kwargs: None
StateUpdateFunction = lambda *args, **kwargs: None

# Concrete
State = lambda *args, **kwargs: None
Parameter = lambda *args, **kwargs: None
Timestep = lambda *args, **kwargs: None
StateHistory = lambda *args, **kwargs: None
Signal = lambda *args, **kwargs: None # Also known as policy_input

### Mapping arguments between user functions and cadCAD objects

An hypothetical application would be to map cadCAD inputs into user inputs. There are a lot of ways of how to do it, and the next block shows an possible choice.


In [3]:
# User function
logistic_map = lambda a, x: (a * x * (1 - x), x)

In [ ]:
# Simulation logic

"""
In this pattern, the policy positional arguments represents the 
generated policy_input object. The argument for signal is the key for it.

As for the keywords arguments, they represent direct mappings into the
callable arguments.
"""
logistic_policy = Policy(logistic_map,
                         Signal('new_population'),
                         Signal('old_population'),
                         a=Parameter('control_parameter'),
                         x=State('population'))

"""
Analogous to the above, but there is only one positional argument,
representing the updated variable.
"""
logistic_update = StateUpdateFunction(lambda x: x,
                                      State('population'),
                                      x=Signal('new_population'))

In [ ]:
# Simulation structure

"""
We could build partial state update blocks in an unordered way
"""
partial_block = PartialStateUpdateBlock(logistic_policy,
                                        logistic_update,
                                        label='Logistic Map Policy')

partial_block_2 = PartialStateUpdateBlock(logistic_update,
                                          label='Nothing at all')

"""
And the timestep block in an ordered manner
"""
timestep_block = TimestepBlock([partial_block, partial_block_2])

In [ ]:
# Simulation parameters
sim_config = {'N': 1, 'T': range(10), 'M': {'control_parameter': 3.7}}
initial_state = {'population': 0.7}

# Wrapping everything
config = Configuration(sim_config,
                       initial_state,
                       timestep_block)


# Run in an simple ExecutionContext
"""
It would be even friendlier if we created an .run() method for running in an very simple context.
Would be very useful for beginners and google colab people.
"""
try:
    data = config.run()
except:
    pass

### Possible applications of using objects for cadCAD objects

* Easier to check relations between the concrete objects, logic and structure
    * Possible optimizations?
    * Easier for generating visualizations of those relations
* More natural for fitting user functions into cadCAD
    * And probably more transparent
* I'm almost sure that this is an pre-requisite for doing class-wrapping functionalities